In [1]:
import os
from glob import glob
from tqdm import tqdm
import pandas as pd
import torch
import torchaudio
from multiprocessing import Pool

from speechbrain.pretrained import EncoderClassifier

/home/jerrymark/data/miniconda3/envs/fs-superb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
metadata = "/home/jerrymark/data/Few-Shot-SUPERB/s3prl/downstream/voxceleb1/veri_test_class"
root = "/home/jerrymark/data/Few-Shot-SUPERB/data/voxceleb1"
output_path = "/home/jerrymark/data/Few-Shot-SUPERB/data/extracted_feats/voxceleb1_x-vector"

In [3]:
df = pd.read_csv(metadata, sep=" ", header=None)

In [4]:
train = df[df[0] ==1]
others = df[df[0] != 1]

In [5]:
train.shape

(13761, 2)

In [5]:
# classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb",run_opts={"device":"cuda"})


In [6]:
# for i, row in tqdm(train.iterrows(), total=len(train)):
#     path = glob(f"{root}/*/wav/{row[1]}")[0]
#     waveform, sample_rate = torchaudio.load(path)
#     feats = classifier.encode_batch(waveform)
#     os.makedirs(os.path.dirname(f"{output_path}/{row[1].replace('.wav', '.pt')}"), exist_ok=True)
#     torch.save(feats, f"{output_path}/{row[1].replace('.wav', '.pt')}")

In [5]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [6]:
sample_list = []
spk_list = glob(f"{output_path}/*")
for spk in tqdm(spk_list):
    samples = sorted(glob(f"{spk}/*/*.pt"))
    X = []
    for sample in samples:
        features = torch.load(sample)[0][0].cpu().numpy()
        X.append(features)

    X = np.array(X)
    X = X / np.linalg.norm(X)

    center = np.mean(X, axis=0)
    distances = np.linalg.norm(X - center, axis=1)
    
    label = pd.qcut(distances, 11, labels=False)
    # c = pd.DataFrame({"path": samples, "label": label})
    # out = c[c.label == 0].sample(1, random_state=8830)
    out = pd.DataFrame({"path": samples, "label": label}).groupby(by="label", group_keys=False).apply(lambda x: x.sample(1, random_state=8830))
    spk_samples = out.path.to_list()
    spk_samples = out.path.apply(lambda x: "/".join(x.split("/")[-3:]).replace(".pt", ".wav")).to_list()
    sample_list.extend(spk_samples)
    

0it [00:00, ?it/s]


In [25]:
metadata_path = "/home/jerrymark/data/Few-Shot-SUPERB/s3prl/downstream/voxceleb1/veri_test_class_1%-special.txt"

In [26]:
with open(metadata_path, "w") as f:
    for sample in sample_list:
        f.write(f"1 {sample}\n")
    for i, row in others.iterrows():
        f.write(f"{row[0]} {row[1]}\n")

In [27]:
# X_embedded = TSNE(n_components=2, learning_rate='auto',
#                 init='random', perplexity=3).fit_transform(X)

# X_embedded = PCA(n_components=2).fit_transform(X)
# plt.scatter(X_embedded[:, 0], X_embedded[:, 1])